# Kategorische Daten

Die meisten Machine Learning Algorithmen können nur mit numerischen Daten umgehen. Bei kategorischen Daten handelt es sich jedoch häufig um Text. Ein Merkmal "Tierart" enthält z.B. die Werte "Hund", "Maus" und "Katze". Ein Merkmal, das den Gemütszustand beschreibt, kann Werte wie z.B. "traurig", "neutral" und "glücklich" enthalten. In beiden Fällen ist eine **Umwandlung in numerische Daten** notwendig. Beim Merkmal "Tierart" handelt es sich um nominale kategorische Daten. Bedeutet, die Werte lassen sich nicht ordnen. Im Gegenteil dazu ist das Merkmal "Gemütszustand" ordinal. Die Werte lassen sich ordnen. Aufsteigend: traurig, neutral, glücklich. 


```{figure} ../images/nominalOrdinal.png
---
height: 200px
align: left
name: fig-nominalOrdinal
---
```

Bei einer Transformation der Merkmalswerte in numerische Daten muss diese Eigenschaft berücksichtigt werden. Im Folgenden werden die Transformationsmethoden für nominale und ordinale Daten behandelt.

In [1]:
import pandas as pd
import numpy as np
import sklearn as sklearn
import pickle

## Nominale Daten

Die einfachste Transformation, von textbasierten zu numerischen kategorischen Merkmalen, ist das Ersetzen einer Kategorie mit einer bestimmten Zahl. Zum Beispiel für "Hund" die 1, für "Maus" die 2 und für "Katze" die 3. Einige ML-Algorithmen gehen jedoch davon aus, dass sich **zwei benachbarte Werte ähnlicher** sind als weiter entfernte Werte. Bei nominalen Daten trifft diese Annahme nicht zu. Es muss sicher gestellt werden, dass der Algorithmus hier **keine Fehlinterpretation** vornimmt. Lösung ist das sogenannte **One-Hot-Encoding**.  

### One-Hot-Encoding

Beim One-Hot-Encoding wird für **jeden möglichen Wert eine Spalte** mit binären Werten erstellt. 1 bedeutet, der Wert liegt vor, 0 bedeutet der Wert liegt nicht vor. Das Merkmal Tierart mit den drei möglichen Kategorien "Hund", "Maus" und "Katze" wird mit dem One-Hot-Encoding in drei Spalten transformiert. Für die Umsetzung bietet sich die [get_dummies()](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)-Methode von Pandas an.

Erstellen und Anzeigen der Beispieldaten:

In [2]:
data = {
    'Tierart': ['Hund', 'Maus', 'Maus', 'Hund', 'Katze', 'Hund'], 
    'Gemütszustand': ['glücklich', 'traurig', 'neutral', 'traurig', 'glücklich', 'glücklich']
}
df = pd.DataFrame(data)
df

,Tierart,Gemütszustand
0,Hund,glücklich
1,Maus,traurig
2,Maus,neutral
3,Hund,traurig
4,Katze,glücklich
5,Hund,glücklich


One-Hot-Encoding des nominalen Merkmal "Tierart".

In [3]:
pd.get_dummies(df['Tierart'])

,Hund,Katze,Maus
0,1,0,0
1,0,0,1
2,0,0,1
3,1,0,0
4,0,1,0
5,1,0,0


## Ordinale Daten

Für ordinale kategorische Daten kann die Transformation durch Ersetzen der Texte mit einer Zahl erfolgen. Eine Möglichkeit der Umsetzung ist der Aufruf der [factorize()](https://pandas.pydata.org/docs/reference/api/pandas.factorize.html)-Methode von Pandas.

In [4]:
df['Gemütszustand'].factorize()

(array([0, 1, 2, 1, 0, 0]),
 Index(['glücklich', 'traurig', 'neutral'], dtype='object'))

Die Liste der Kategorien gibt an wie das Mapping stattgefunden hat. "glücklich" wurde mit der Zahl 0 ersetzt, "traurig" mit der Zahl 1 und "neutral" mit der Zahl 2. Jetzt würden die meisten Machine Learning Algorithmen interpretieren dass "glücklich" ähnlicher "traurig" ist als "neutral". Die einfach anzuwendene factorize()-Methode kann funktionieren, muss aber nicht. **Kontrollieren** Sie nach der Anwendung ob die Zahlenwerte auch der **Reihenfolge der Ordnung** entsprechen. Alternativ kann die [replace()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)-Methode verwendet und das Mapping **explizit** bestimmt werden.

In [5]:
mapping = {
    "glücklich": 0,
    "neutral": 1,
    "traurig": 2
}

In [6]:
df['Gemütszustand'].replace(mapping)

0    0
1    2
2    1
3    2
4    0
5    0
Name: Gemütszustand, dtype: int64

```{important}
Die Ordnung muss sich in den zugewiesenen Zahlen wiederfinden. 
````

## Transformation kategorischer Merkmale am Beispiel Titanic

Laden der Datensets aus Pickle File.

In [7]:
with open('../output/titanic/datasets_or.pkl', 'rb') as handle:
    datasets = pickle.load(handle)

Die verbleibenden kategorischen Merkmale des Titanic Datensets sind die Ticketklasse (Pclass), das Geschlecht (Sex) und der Zustiegsort (Embarked). 

In [8]:
X_train = datasets['X_train'].copy()
X_train[['Pclass', 'Sex', 'Embarked']]

,Pclass,Sex,Embarked
109,3,female,Q
391,3,male,S
6,1,male,S
405,2,male,S
291,1,female,C
...,...,...,...
79,3,female,S
527,1,male,S
536,1,male,S
742,1,female,C


### Ticketklasse

Bei der Ticketklasse handelt es sich um ordinale Daten: 1 ist die höchste Klasse, 2 die mittlere und 3 die niedrigste Klasse. Über das Attribut dtype kann der Datentyp abgefragt werden. 

In [9]:
X_train['Pclass'].dtype

dtype('int64')

Es handelt sich bereits um Integer Werte. In diesem Fall muss keine Transformation vorgenommen werden.

### Gechlecht

Das Merkmal Geschlecht enthält nominale Kategorien. Es wird das One-Hot Encoding angewendet.

In [10]:
pd.get_dummies(X_train['Sex'])

,female,male
109,1,0
391,0,1
6,0,1
405,0,1
291,1,0
...,...,...
79,1,0
527,0,1
536,0,1
742,1,0


### Zustiegsort

Der Zustiegsort enthält ebenso nominale Kategorien. Es wird das One-Hot Encoding angewendet.

In [11]:
pd.get_dummies(X_train['Embarked'])

,C,Q,S
109,0,1,0
391,0,0,1
6,0,0,1
405,0,0,1
291,1,0,0
...,...,...,...
79,0,0,1
527,0,0,1
536,0,0,1
742,1,0,0


Sie kennen nun die wichtigsten Methoden um numerische und kategorische Daten für Machine Learning Algorithmen aufzubereiten. Im nächsten Abschnitt wird gezeigt, wie die Transformationen angewendet werden. 